In [0]:
BEGIN ATOMIC

    -- Statement 1: Insert morning activity into metrics table
    INSERT INTO iot_device_metrics
    SELECT
      device_id,
      user_id,
      CAST(timestamp AS TIMESTAMP) AS timestamp,
      num_steps,
      calories_burnt,
      miles_walked
    FROM iot_device_raw
    WHERE device_id IS NOT NULL
      AND num_steps > 0
      AND DATE(CAST(timestamp AS TIMESTAMP)) = '2018-07-20'
      AND HOUR(CAST(timestamp AS TIMESTAMP)) BETWEEN 6 AND 8;

    -- Statement 2: Update daily summary table
    MERGE INTO iot_daily_summary AS target
    USING (
      SELECT
        device_id,
        user_id,
        DATE(timestamp) AS date,
        SUM(num_steps) AS total_steps,
        SUM(miles_walked) AS total_miles,
        SUM(calories_burnt) AS total_calories
      FROM iot_device_metrics
      WHERE DATE(timestamp) = '2018-07-20'
        AND HOUR(timestamp) BETWEEN 6 AND 8
      GROUP BY device_id, user_id, DATE(timestamp)
    ) AS source
    ON target.device_id = source.device_id
      AND target.user_id = source.user_id
      AND target.date = source.date
    WHEN MATCHED THEN UPDATE SET
      target.total_steps = source.total_steps,
      target.total_miles = source.total_miles,
      target.total_calories = source.total_calories
    WHEN NOT MATCHED THEN INSERT *;

    -- Statement 3: Create alerts in alerts table
    INSERT INTO iot_alerts (device_id, alert_type, alert_message, created_at)
    SELECT
      device_id,
      'MORNING_ACTIVITY' AS alert_type,
      CONCAT('Device ', device_id, ' logged ', total_steps, ' steps on July 20th (6-9 AM)') AS alert_message,
      current_timestamp() AS created_at
    FROM iot_daily_summary
    WHERE date = '2018-07-20'
      AND total_steps > 500;

  END;
